# Configuración de un Clúster HDFS en Docker y Operaciones Básicas en HDFS

Paso 0: Crear el directorio del proyecto. Si estas trabajando desde vs code con wsl es posible que al momento de crear los ficheros tengas problemas con los permisos. Normalmente la ruta desd wsl empieza por /mnt por lo que debemos movernos a home/usuario. 

Crea un directorio para el proyecto y navega a él.

In [ ]:
mkdir /home/balrodjuan/hdfs-docker-cluster
cd /home/balrodjuan/hdfs-docker-cluster

### Paso 1: Crear la estructura de directorios

Ejecuta los siguientes comandos para crear la estructura de directorios necesaria.

In [ ]:
mkdir -p hadoop_config
mkdir hadoop_namenode hadoop_datanode1 hadoop_datanode2
touch docker-compose.yml hadoop_config/core-site.xml hadoop_config/hdfs-site.xml hadoop_config/log4j.properties start-hdfs.sh init-datanode.sh

La estructura deberia verse asi:

In [ ]:
hdfs-docker-cluster/
│
├── docker-compose.yml
├── hadoop_config/
│   ├── core-site.xml
│   ├── hdfs-site.xml
│   ├── log4j.properties
├── start-hdfs.sh
├── init-datanode.sh
├── hadoop_namenode/
├── hadoop_datanode1/
└── hadoop_datanode2/

### Paso 2: Configuración de Docker Compose

2.1 Abre el archivo `docker-compose.yml` y agrega el siguiente contenido:

In [ ]:
services:
  namenode:
    image: apache/hadoop:3.3.5
    container_name: namenode
    hostname: namenode
    user: root
    environment:
      - HADOOP_HOME=/opt/hadoop
    volumes:
      - ./hadoop_namenode:/opt/hadoop/data/nameNode
      - ./hadoop_config:/opt/hadoop/etc/hadoop
      - ./start-hdfs.sh:/start-hdfs.sh
    ports:
      - "9870:9870"
    command: [ "/bin/bash", "/start-hdfs.sh" ]
    networks:
      hdfs_network:
        ipv4_address: 172.20.0.2

  datanode1:
    image: apache/hadoop:3.3.5
    container_name: datanode1
    hostname: datanode1
    user: root
    environment:
      - HADOOP_HOME=/opt/hadoop
    volumes:
      - ./hadoop_datanode1:/opt/hadoop/data/dataNode
      - ./hadoop_config:/opt/hadoop/etc/hadoop
      - ./init-datanode.sh:/init-datanode.sh
    depends_on:
      - namenode
    command: [ "/bin/bash", "/init-datanode.sh" ]
    networks:
      hdfs_network:
        ipv4_address: 172.20.0.3

  datanode2:
    image: apache/hadoop:3.3.5
    container_name: datanode2
    hostname: datanode2
    user: root
    environment:
      - HADOOP_HOME=/opt/hadoop
    volumes:
      - ./hadoop_datanode2:/opt/hadoop/data/dataNode
      - ./hadoop_config:/opt/hadoop/etc/hadoop
      - ./init-datanode.sh:/init-datanode.sh
    depends_on:
      - namenode
    command: [ "/bin/bash", "/init-datanode.sh" ]
    networks:
      hdfs_network:
        ipv4_address: 172.20.0.4

networks:
  hdfs_network:
    ipam:
      driver: default
      config:
        - subnet: 172.20.0.0/16

### Paso 3: Crear los scripts de inicialización

Script `start-hdfs.sh` para el NameNode. Agrega el siguiente contenido al archivo `start-hdfs.sh`:

In [ ]:
#!/bin/bash
if [ ! -d "/opt/hadoop/data/nameNode/current" ]; then
    echo "Formatting NameNode..."
    hdfs namenode -format
fi
hdfs namenode

Script `init-datanode.sh` para los DataNodes. agrega el siguiente contenido al archivo `init-datanode.sh`:

In [ ]:
#!/bin/bash
rm -rf /opt/hadoop/data/dataNode/*
chown -R hadoop:hadoop /opt/hadoop/data/dataNode
chmod 755 /opt/hadoop/data/dataNode
hdfs datanode

Dar permisos de ejecución a los scripts

In [ ]:
chmod +x start-hdfs.sh
chmod +x init-datanode.sh

### Paso 4: Configuración de los archivos de Hadoop

Configuración de `core-site.xml`. Agrega el siguiente contenido al archivo `core-site.xml`:

In [ ]:
<configuration>
  <property>
    <name>fs.defaultFS</name>
    <value>hdfs://namenode:8020</value>
  </property>
</configuration>

Configuración de `hdfs-site.xml`. Agrega el siguiente contenido al archivo `hdfs-site.xml`:

In [ ]:
<configuration>
  <property>
    <name>dfs.replication</name>
    <value>2</value>
  </property>
  <property>
    <name>dfs.namenode.name.dir</name>
    <value>/opt/hadoop/data/nameNode</value>
  </property>
  <property>
    <name>dfs.datanode.data.dir</name>
    <value>/opt/hadoop/data/dataNode</value>
  </property>
</configuration>

### Paso 5: Configuración de `log4j.properties`.

Para evitar la advertencia de log4j, agrega la siguiente configuración básica:

In [ ]:
log4j.rootLogger=INFO, console
log4j.appender.console=org.apache.log4j.ConsoleAppender
log4j.appender.console.target=System.err
log4j.appender.console.layout=org.apache.log4j.PatternLayout
log4j.appender.console.layout.ConversionPattern=%d{ISO8601} %-5p %c{2} - %m%n


### Paso 6. Lanzar el clúster

Inicia el clúster de HDFS usando Docker Compose:

In [ ]:
docker-compose up -d

Verifica que los contenedores estén en funcionamiento:

In [ ]:
docker ps

Accede a la interfaz web del NameNode en: `http://localhost:9870`

### Paso 7. Operaciones Básicas en HDFS

7.1 Listar Directorios y Archivos en HDFS. Para ver el contenido de un directorio en HDFS:

In [ ]:
docker exec -it namenode hdfs dfs -ls /

7.2 Subir Archivos a HDFS. Crear un archivo de prueba:

In [ ]:
echo "Hello, HDFS" > test.txt

7.3 Copiar el archivo al contenedor `namenode`:

In [ ]:
docker cp test.txt namenode:/tmp/

7.4 Subir el archivo a HDFS:

In [ ]:
docker exec -it namenode hdfs dfs -put /tmp/test.txt /

7.5  Ver el Contenido de un Archivo en HDFS

In [ ]:
docker exec -it namenode hdfs dfs -cat /test.txt

7.6 Eliminar un Archivo o Directorio en HDFS

In [ ]:
docker exec -it namenode hdfs dfs -rm /test.txt

7.7 Eliminar un directorio y su contenido

In [ ]:
docker exec -it namenode hdfs dfs -rm -r /ruta/en/hdfs/directorio

7.8 Crear un Directorio en HDFS

In [ ]:
docker exec -it namenode hdfs dfs -mkdir /ruta/en/hdfs/nuevo_directorio

Source: https://bytemedirk.medium.com/setting-up-an-hdfs-cluster-with-docker-compose-a-step-by-step-guide-4541cd15b168